In [8]:
def live_combined(state_widget, model_steering, model_obstacle, model_sign, camera, prediction_widget):
    global threshold_blocked, threshold_speedup
    while state_widget.value == 'live':
        image = camera.value
        img_proc = preprocess(image)

        # Dự đoán vật cản (ví dụ model output logits 2 lớp: blocked, free)
        output_obstacle = model_obstacle(img_proc)
        prob = torch.softmax(output_obstacle, dim=1)
        prob_blocked = prob[0, 0].item()  # giả sử class 0 = blocked

        # Dự đoán góc lái
        output_steer = model_steering(img_proc)
        steer = output_steer.item()  # giả sử output 1 giá trị steering

        # Dự đoán biển báo
        output_sign = model_sign(img_proc)
        pred_sign = torch.argmax(output_sign, 1).item()
        sign_label = model_sign_categories[pred_sign]  # vd ['none', 'min_30']

        # Logic điều khiển
        if prob_blocked < threshold_blocked:
            # Không có vật cản, xử lý biển báo
            if sign_label == 'min_30':
                throttle = throttle_fast
            else:
                throttle = throttle_normal
            steering = steer * STEERING_GAIN + STEERING_BIAS
        else:
            throttle = 0.0
            steering = 0.0

        # Cập nhật xe
        car.throttle = throttle
        car.steering = steering

        # Hiển thị ảnh dự đoán có vòng tròn, biển báo, thông tin
        h, w, _ = image.shape
        cx = int(w * 0.5)
        cy = int(h * 0.8)
        pred_img = image.copy()
        cv2.putText(pred_img, f"Blocked: {prob_blocked:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        cv2.putText(pred_img, f"Steer: {steering:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)
        cv2.putText(pred_img, f"Sign: {sign_label}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        prediction_widget.value = bgr8_to_jpeg(pred_img)


NameError: name '_C' is not defined

In [ ]:
threshold_blocked = 0.5
throttle_normal = 0.18
throttle_fast = 0.3
STEERING_GAIN = 0.9
STEERING_BIAS = 0.0

model_sign_categories = ['none', 'min_30']  # ví dụ


In [ ]:
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

def start_live(change):
    if change['new'] == 'live':
        threading.Thread(target=live_combined, args=(state_widget, model_steering, model_obstacle, model_sign, camera, prediction_widget)).start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([prediction_widget, state_widget])
display(live_execution_widget)


In [ ]:
def preprocess(image):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0).to(device)
